<a href="https://colab.research.google.com/github/anh1811/baocao/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import tensorflow as tf
import numpy as np
import scipy.optimize as opt

SyntaxError: ignored

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")

In [ ]:
x_train[2,:][np.newaxis,:][0,:].shape

(28, 28)

In [ ]:
x_train = x_train.reshape(x_train.shape[0],-1)
y_train = y_train
x_val = x_test[0:5000].reshape(5000,-1)
y_val = y_test[0:5000]
x_test = x_test[5000:10000].reshape(5000,-1)
y_test = y_test[5000:10000]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-x));
def sigmoidGradient(x):
  return sigmoid(x)*(1-sigmoid(x))

In [ ]:
def nnCostFunction1(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda):
  Theta1 = nn_params[0:hidden_layer_size * (input_layer_size + 1)].reshape(\
                          hidden_layer_size, (input_layer_size + 1))
  Theta2 = nn_params[hidden_layer_size * (input_layer_size + 1):].reshape(\
                          num_labels, (hidden_layer_size + 1))
  I = np.eye(num_labels)
  X = np.append(1,X) #333
  z2 = np.dot(X,Theta1.T) # X * theta1' (m,hiddenlayersize)
  a2 = sigmoid(z2)
  a2 = np.append(1,a2) # m * hiddenlayersize + 1
  z3 = np.dot(a2,Theta2.T) # m * 10
  a3 = sigmoid(z3) #10
  J = 0
  Theta1_grad = np.zeros((hidden_layer_size,input_layer_size+1))
  Theta2_grad = np.zeros((num_labels,hidden_layer_size+1))
  y = I[int(y)]
  J = - np.dot(np.log10(a3[np.newaxis,:]),y) - (np.log10((1 - a3)[np.newaxis,:]) @ (1 -y))
  delta3 = a3 - y
  delta2 = np.dot(Theta2.T,delta3) * sigmoidGradient(np.append(1,z2))  #(hls+1,)
  delta2 = delta2[1:] #(hls,)
  Theta1_grad += np.dot(delta2[:,np.newaxis],X[np.newaxis,:]) # hls, ils+1
  Theta2_grad += np.dot(delta3.reshape(num_labels,1),a2.reshape(1,hidden_layer_size+1)) # 10,hls+1

  J += Lambda/2*(np.sum(Theta1[:,1:]**2) + np.sum(Theta2[:,1:]**2))
  Theta1_grad = Theta1_grad 
  Theta1_grad[:,1:] += Lambda * Theta1[:,1:]
  Theta2_grad = Theta2_grad 
  Theta2_grad[:,1:] += Lambda * Theta2[:,1:]

  grad = np.append(Theta1_grad, Theta2_grad)
  return [J,grad]

In [ ]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda):
  Theta1 = np.reshape(nn_params[0:hidden_layer_size * (input_layer_size + 1)],\
                          [hidden_layer_size, (input_layer_size + 1)])
  Theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):],\
                          [num_labels, (hidden_layer_size + 1)])
  m = X.shape[0];
  I = np.eye(num_labels)
  X = np.concatenate((np.ones((m,1)),X),1) 
  z2 = np.dot(X,Theta1.T) 
  a2 = sigmoid(z2)
  a2 = np.concatenate((np.ones((m,1)),a2),1) 
  z3 = np.dot(a2,Theta2.T) 
  a3 = sigmoid(z3)
  J = 0
  Theta1_grad = np.zeros((hidden_layer_size,input_layer_size+1))
  Theta2_grad = np.zeros((num_labels,hidden_layer_size+1))
  for i in range(0,m):
    yi = I[int(y[i])] 
    J += (-np.dot(np.log10(a3[i,:]),yi) - np.log10((1 - a3[i,:])) @ (1 -yi)) * 1/m
    delta3 = a3[i,:] - yi
    delta2 = np.dot(Theta2.T,delta3) * sigmoidGradient(np.append(1,z2[i,:])) 
    delta2 = delta2[1:] 
    Theta1_grad += np.dot(delta2[:,np.newaxis],X[i,:][np.newaxis,:])
    Theta2_grad += np.dot(delta3.reshape(num_labels,1),a2[i,:].reshape(1,hidden_layer_size + 1))
  J += Lambda/(2*m)*(np.sum(Theta1[0:,1:]**2) + np.sum(Theta2[0:,1:]**2))
  Theta1_grad = Theta1_grad/m 
  Theta1_grad[0:,1:] += Lambda/m * Theta1[0:,1:]
  Theta2_grad = Theta2_grad/m 
  Theta2_grad[0:,1:] += Lambda/m * Theta2[0:,1:]

  grad = np.append(Theta1_grad, Theta2_grad)
  return [J,grad]



def randInitializeWeights(L_in, L_out):
  epsilon_init = np.sqrt(6)/np.sqrt(L_in+L_out)
  W = np.random.random((L_out, 1+L_in))*(2*epsilon_init) - epsilon_init
  return W

def featureNormalize(X):
  mu = np.mean(X,axis =0)
  X_norm = X-mu
  sigma = np.std(X_norm,axis =0, ddof=1)
  sigma_1 = np.where(sigma==0,1,sigma)
  X_norm = X_norm/sigma_1
  return X_norm, mu, sigma

def predict(Theta1, Theta2, X):

  m = X.shape[0];
  num_labels = Theta2.shape[0];

  h1 = sigmoid(np.concatenate((np.ones((m,1)),X),1) @ Theta1.T)
  h2 = sigmoid(np.concatenate((np.ones((m,1)),h1),1) @ Theta2.T)

  p = np.argmax(h2,axis=1)

  return p

def pca(X):
  m = X.shape[0];
  Sigma = (1/m)*np.dot(X.T,X)
  U, S, Vh = np.linalg.svd(Sigma)
  return (U,S)

def projectData(X, U, K):
  Ureduce = U[:,0:K]
  Z = np.dot(X,Ureduce)
  return Z

   

In [ ]:
x_train, mu_train, sigma_train = featureNormalize(x_train)
x_test, mu_test, sigma_test =featureNormalize(x_test)
x_val, mu_val, sigma_val = featureNormalize(x_val)
print("Normalize Data! Done!")

Normalize Data! Done!


In [ ]:
K = 1
var = 0
U,S = pca(x_train)
S_sum = S.sum() 
while(var < 0.95):
  var = S[0:K].sum()/S_sum
  K += 1
  print(var)
print(K) 

0.05646716919741764
0.09724988908210876
0.1346436932980204
0.16349484175413345
0.18870592801066705
0.21064862765748582
0.22988207152080636
0.24734006380483683
0.2626909868507977
0.2767081828674872
0.2901256130661941
0.3021630324608486
0.31330872795849646
0.3242079635196384
0.3344944557556942
0.344439321393158
0.35380315419525726
0.3630136108505784
0.3719479786271314
0.38064710481471714
0.38892073500511937
0.3969549086989737
0.4046033637025235
0.4120210883417664
0.4191740170214982
0.426092485336152
0.4329338449752703
0.43950059043741635
0.44581735767560526
0.4519465560669777
0.4579091090207258
0.4637862731772543
0.4695021901693696
0.4751252643341362
0.4806720843583386
0.4860562681005487
0.49136809059713804
0.49656414661403103
0.5016462591601256
0.5064463148680689
0.5112108730680837
0.5159022666681029
0.5204457562288387
0.5249592140941933
0.529428848105275
0.5338626796596289
0.5382448343446772
0.5425486518569812
0.5468174308633516
0.5510539010352965
0.5551008622486074
0.5590953362835827


In [ ]:
# Found from above algorithm
x_train = projectData(x_train, U, K)
x_test = projectData(x_test,U,K)
x_val = projectData(x_val,U,K)

In [ ]:
X = np.concatenate((x_train,y_train[:,np.newaxis]), 1)
np.random.shuffle(X)
x_train = X[:,:-1]
y_train = X[:,-1]

In [ ]:
I = np.eye(num_labels)
I[int(y_train[100])] 

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [ ]:
input_layer_size  = K
hidden_layer_size = 64
num_labels = 10


print('\nInitializing Neural Network Parameters ...\n')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)
initial_nn_params = np.append(initial_Theta1,initial_Theta2)

# num_iters = 150
# j_avg = 1

# Lambda_vec = [0.3,1, 3]
nn_params = initial_nn_params
#Lambda_vec_len = len(Lambda_vec)
val_result_max =0 #Will be changed later

m = x_train.shape[0]
for j in range(10):
  for i in range(m):
    J,grad = nnCostFunction1(nn_params, input_layer_size, hidden_layer_size, num_labels, x_train[i,:], y_train[i], 0.001)
    nn_params -= 0.1*grad
    i += 1
  j += 1


Theta1 = np.reshape(nn_params[0:hidden_layer_size * (input_layer_size + 1)],\
                           [hidden_layer_size, (input_layer_size + 1)])
Theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):],\
                           [num_labels, (hidden_layer_size + 1)])
pred_val = predict(Theta1, Theta2, x_val);
val_result = (np.mean(pred_val==y_val) * 100)
pred_train = predict(Theta1, Theta2, x_train);
train_result = (np.mean(pred_train==y_train) * 100)
#   print(np.mean(pred_val==y_val) * 100)


# for i in range(1):
#   print(f"With Lambda = {Lambda_vec[1]}, {num_iters} iters")
#   val_result =0

#   result = opt.minimize(nnCostFunction, x0=initial_nn_params, args=(\
#         input_layer_size, hidden_layer_size, num_labels, x_train, y_train, Lambda_vec[1])\
#         ,method='L-BFGS-B',jac=True,options={'maxiter':num_iters})
      
#   nn_params = result.x


#   Theta1 = np.reshape(nn_params[0:hidden_layer_size * (input_layer_size + 1)],\
#                           [hidden_layer_size, (input_layer_size + 1)])
#   Theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):],\
#                           [num_labels, (hidden_layer_size + 1)])
#   pred_val = predict(Theta1, Theta2, x_val);

#   val_result = (np.mean(pred_val==y_val) * 100)
#   print(np.mean(pred_val==y_val) * 100)


#   if val_result >val_result_max:
#     val_result_max = val_result
#     Lambda = Lambda_vec[i]
#     final_nn_params = nn_params


print(f"The accuracy on validate set is about {val_result}%\n")
print(f"The accuracy on train set is about {train_result}%\n")



Initializing Neural Network Parameters ...



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


The accuracy on validate set is about 89.92%

The accuracy on train set is about 92.56166666666667%



In [ ]:
np.isscalar(y_test[1])

True

In [ ]:
# # J = np.save(r"/content/drive/MyDrive/MNIST problem",final_nn_params,allow_pickle = False)
# nn_params = np.load(r"/content/drive/MyDrive/MNIST problem.npy")
Theta1 = np.reshape(nn_params[0:hidden_layer_size * (input_layer_size + 1)],\
                        [hidden_layer_size, (input_layer_size + 1)])
Theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):],\
                        [num_labels, (hidden_layer_size + 1)])

In [ ]:
pred_train = predict(Theta1, Theta2, x_train);
print(f"The accuracy on train set is about {np.mean(pred_train==y_train) * 100}%")

pred_test = predict(Theta1, Theta2, x_test);
print(f"The accuracy on test set is about {np.mean(pred_test==y_test) * 100}%")

The accuracy on train set is about 11.236666666666666%
The accuracy on test set is about 11.28%


In [ ]:
import pandas as pd
X = np.array(pd.read_csv("/content/sample_data/mnist_test.csv")) 

In [ ]:
X.shape

(9999, 785)

In [ ]:
x = np.array(X[:,1:])
y = np.array(X[:,0])
x, mu, sigma = featureNormalize(x)
x = projectData(x, U, K)
y

array([2, 1, 0, ..., 4, 5, 6])

In [ ]:
pre = predict(Theta1,Theta2,x);
print(f"The accuracy on test set is about {np.mean(pre==y) * 100}%")

The accuracy on test set is about 95.6095609560956%
